# Combining Datasets: Concat and Append

Здесь мы рассмотрим простое объединение объектов `Series` и `DataFrame` с помощью функции `pd.concat`; в дальнейшем мы рассмотрим более сложные объединения в памяти и соединения доступные в Pandas.

In [2]:
import pandas as pd
import numpy as np

Для удобства вы определим функцию, которая будет создавать объекты `DataFrame` определённой формы:

In [4]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


В дополнение мы создадим класс, который позволит нам отображать несколько объектов `DataFrame` друг против друга. Код использует специальный метод `_repr_html_`, который используется в `IPython` для расширенного отображения объектов:

In [5]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_()) for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a)) for a in self.args)

## Напоминание: объединение массивов NumPy

Объединение объектов `Series` и `DataFrame` очень похоже на объединение массивов NumPy, которое может быть выполнено через функцию `np.concatenate` как это было рассмотрено [The Basics of NumPy Arrays](https://jakevdp.github.io/PythonDataScienceHandbook/02.02-the-basics-of-numpy-arrays.html). Например:

In [7]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

Вы также можете указать ось, по которой будет проходить объединение:

In [11]:
x = [[1, 2], [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## Простое объединение с помощью `pd.concat`

Функция `pd.concat` имеет схожий с `np.concatenate` синтакс, но содержит дополнительные опции:

```python
# Signature in Pandas v2.2
pandas.concat(
    objs,
    *,
    axis=0,
    join='outer',
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    sort=False,
    copy=None
)
```

`pd.concat` может быть использована для объединения объектов `Series` или `DataFrame`:

In [15]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

`pd.concat` также может быть использована для объединения `DataFrame`:

In [17]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


По умолчанию, внутри `DataFrame` объединение выполняется по рядам (т.е. `axis=0`). Как и `np.concatenate`, `pd.concat` позволяет указать ось, по которой будет выполняться объединение. Давайте возьмем следующий пример:

In [28]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='columns')")
# можно также задать ось с помощью числа
# pd.concat([df3, df4], axis=1)

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis='columns')
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

### Дубликаты индексов

Одна важная разница между `np.concatenate` и `pd.concat` состоит в том, что объединение в Pandas сохраняет индексы, даже если результат будет содержать одинаковые индексы! Возьмем следующий пример:

In [29]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Обратите внимение на повторяющиеся индексы, в то время как такое результат является правильным, он часто не является желаемым. `pd.concat` даёт нам несколько возможностей, чтобы обработать такие ситуации.

#### Генерация ошибок при повторениях

Если вы просто хотите убедиться, что, в результате вывода `pd.concat`, нет повторений, вы можете просто установить флаг `verify_integrity`. Когда он установлен в `True`, объединение вызовет исключение при наличии дубликатов в индексах. Например:

In [32]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Index([0, 1], dtype='int64')


#### Игнорирование индексов

Иногда сам по себе индекс не является настолько важным, и вы могли бы предпочесть просто проигнорировать его. Такая возможность может быть включена с помощью флага `ignore_index`. Когда этот флаг установлен в `True`, объединение создать новый целочисленный индекс для выходного объекта `Series`:

In [33]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### Добавление мультииндексных ключей

Другая возможность состоит в использовании опции `keys`, для указания меток для источника данных; результат будет содержать иерархический индекс:

In [35]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

### Объединение с помощью соединений

В простых примерах, которые мы только что рассмотрели, мы, главным образом, объединяли объекты `DataFrame` содержащие одинаковые имена колонок. На практике, данные из разных источников могут содержать различные наборы имён колонок, и `pd.concat` предоставляет несколько опций для данного случая. Например, рассмотрим объединение двух `DataFrame`, которые имеют некоторые (но не все!) общие колонки:

In [37]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

По умолчанию, записи, для которых данные недоступны, заполняются `NaN` значениями. Для изменения этого, мы можем указать одну или несколько опция для параметров `join` и `join_axes`. По умолчиню, соединение является объединением входных колонок (`join='outer'`), но, с помощью `join='inner'` мы можем изменить это на пересечение колонок:

In [39]:
# join='inner'
display('df5', 'df6', "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

ЭТА ВОЗМОЖНОСТЬ ОТСУТСТВУЕТ В ТЕКУЩЕМ PANDAS!!!

Другая опция состоит в прямом указании индекса оставляемых колонок с помощью аргумента `join_axes`, который принимает список индексных объектов. Здесь мы укажем, что возвращаемые колонки должны в точности соответствовать колонкам первого объекта:

In [42]:
# display('df5', 'df6', "pd.concat([df5, df6], join_axes=[df5.columns])")

### Метод `_append()`

Ввиду того, что прямое объединение является очень популярным, объекты `Series` и `DataFrame` имеют метод `append`, который позволяет достичь точно такого же результата с меньшим объемом кода. Например, вместо вызова `pd.concat([df1, df2])` вы можете просто сделать так `df1._append(df2)`:

In [45]:
display('df1', 'df2', 'df1._append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


В оригинале указан метод `append()`, но такой метод отсутствует в классах `Series` и `DataFrame`. Метод `_append()` доступен на данный момент, но наличие подчеркивания говорит о не слишком высокой целесообразности его использования.